<a href="https://colab.research.google.com/github/Showmick119/Samay/blob/main/moment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Dependencies**

In [ ]:
!pip install samay-0.1.0-cp311-cp311-linux_x86_64.whl

In [ ]:
!pip install --upgrade -U numpy --force

# **Loading Moment Model**

In [ ]:
from samay.model import MomentModel
from samay.dataset import MomentDataset
from samay.models.moment.momentfm.models.statistical_classifiers import fit_svm
from samay.utils import load_args

repo = "AutonLab/MOMENT-1-large"
config = {"task_name": "classification", "n_channels": 1, "num_class": 5}
mmt = MomentModel(config=config, repo=repo)

# **Loading Dataset**

In [ ]:
!wget https://raw.githubusercontent.com/kage08/Samay/refs/heads/main/data/data/ETTh1.csv

# **Use a SVM classifier to evaluate zero-shot embeddings of Moment model**

In [ ]:
train_dataset = MomentDataset(
    name="ecg5000",
    path="/nethome/sli999/TSFMProject/src/tsfmproject/models/moment/data/ECG5000_TRAIN.csv",
    batchsize=64,
    mode="train",
    task_name="classification",
)
test_dataset = MomentDataset(
    name="ecg5000",
    path="/nethome/sli999/TSFMProject/src/tsfmproject/models/moment/data/ECG5000_TEST.csv",
    batchsize=64,
    mode="test",
    task_name="classification",
)

train_accuracy, train_embeddings, train_labels = mmt.evaluate(
    train_dataset, task_name="classification"
)
test_accuracy, test_embeddings, test_labels = mmt.evaluate(
    test_dataset, task_name="classification"
)
print(train_embeddings.shape, train_labels.shape)

clf = fit_svm(features=train_embeddings, y=train_labels)

y_pred_train = clf.predict(train_embeddings)
y_pred_test = clf.predict(test_embeddings)
train_accuracy = clf.score(train_embeddings, train_labels)
test_accuracy = clf.score(test_embeddings, test_labels)

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")

# **Visualize the embeddings**

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


test_embeddings_manifold = PCA(n_components=2).fit_transform(test_embeddings)

plt.title("ECG5000 Test Embeddings", fontsize=20)
plt.scatter(
    test_embeddings_manifold[:, 0],
    test_embeddings_manifold[:, 1],
    c=test_labels.squeeze(),
)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.show()

# **Finetune Moment model for classification**

In [ ]:
finetuned_model = mmt.finetune(
    train_dataset, task_name="classification", epoch=10, lr=0.1
)

# **Evaluate the finetuned model**

In [ ]:
accuracy, embeddings, lebels = mmt.evaluate(test_dataset, task_name="classification")
print(accuracy)